In [2]:
pip install ragas

  Obtaining dependency information for ragas from https://files.pythonhosted.org/packages/b6/1f/1087efbd0d0723ef8212aba2dfd035bdbcef6698623b29e6f724ad8cdcf9/ragas-0.2.13-py3-none-any.whl.metadata
  Obtaining dependency information for datasets from https://files.pythonhosted.org/packages/d7/84/0df6c5981f5fc722381662ff8cfbdf8aad64bec875f75d80b55bfef394ce/datasets-3.2.0-py3-none-any.whl.metadata
  Obtaining dependency information for tiktoken from https://files.pythonhosted.org/packages/45/e2/39d4aa02a52bba73b2cd21ba4533c84425ff8786cc63c511d68c8897376e/tiktoken-0.8.0-cp312-cp312-win_amd64.whl.metadata
  Obtaining dependency information for langchain from https://files.pythonhosted.org/packages/e9/65/e5cc2876078fa5f1a621c8429f0174855c7e9831060d350626dbf8d2a10c/langchain-0.3.17-py3-none-any.whl.metadata
  Obtaining dependency information for langchain-core from https://files.pythonhosted.org/packages/98/78/463bc92174555cc04b3e234faa169bb8b58f36fff77892d7b8ae2b4f58e4/langchain_core-0.3.33-p


[notice] A new release of pip is available: 23.2.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
pip install git+https://github.com/explodinggradients/ragas.git

  Cloning https://github.com/explodinggradients/ragas.git to c:\users\김재빈\appdata\local\temp\pip-req-build-o7ot_vb_
  Resolved https://github.com/explodinggradients/ragas.git to commit 2605de5224a7c0e90f000d74f561d2dae770c18f
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
Note: you may need to restart the kernel to use updated packages.


  Running command git clone --filter=blob:none --quiet https://github.com/explodinggradients/ragas.git 'C:\Users\김재빈\AppData\Local\Temp\pip-req-build-o7ot_vb_'

[notice] A new release of pip is available: 23.2.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [13]:
import os
from dotenv import load_dotenv
from openai import AzureOpenAI

load_dotenv()
azure_endpoint = os.getenv('AZURE_OPENAI_ENDPOINT')
azure_api_key = os.getenv('AZURE_OPENAI_API_KEY')

client = AzureOpenAI(
        azure_endpoint = azure_endpoint, 
        api_key = azure_api_key,  
        api_version = "2024-02-01"
    )

response = client.chat.completions.create(
    model="gpt-4o-doc-search",
    messages=[{"role": "system", "content":'Your task is to judge the faithfulness of a series of statements based on a given context. For each statement you must return verdict as 1 if the statement can be directly inferred based on the context or 0 if the statement can not be directly inferred based on the context.'}]
)

evaluator_llm = response

In [16]:
from ragas import SingleTurnSample 
from ragas.metrics import Faithfulness

# 평가할 샘플 구성 (참조 문서 내용 포함)
sample = SingleTurnSample(
    user_input="복막 투석 수술을 할건데 흔히 생기는 합병증이 있어?",
    response="""복막투석에서 흔히 발생할 수 있는 합병증으로는 다음과 같은 문제들이 있습니다:

복막염: 복막투석 중 가장 흔한 합병증으로, 투석 과정 중 감염이 발생할 수 있습니다. 이는 투석액이 탁하게 보이거나 복통, 발열 등의 증상으로 나타날 수 있습니다 ^1^ ^2^ .
도관 관련 합병증: 복막투석 도관 삽입 후, 도관 주위 감염이나 도관 기능 부전 등의 문제가 발생할 수 있습니다 ^3^ .
기타 합병증: 복막 기능 저하나 유착, 그리고 복막을 통한 투석액 흡수로 인한 영양 불균형 등이 포함될 수 있습니다 ^3^ .

복막투석을 준비하거나 시행 중이라면, 감염 예방 및 정기적인 상태 점검이 중요합니다. 의료진의 지시에 따라 관리하면 합병증 위험을 줄일 수 있습니다.""",
    retrieved_contexts=[
        """63-복막투석.pdf
가장 흔한 합병증으로 복막염이 있습니다

복막 투석을 계속하지 못하게 되는 가장 흔한 원인이 되며, CAPD 환자가 입원하는 
가장 흔한 원인이기도 합니다..복막 유착 및 복막표면적의 상실로 부적절한 투석

라..투석 중 저혈압이 발생할 수 있습니다..서명 : (인)

이 수술 이외의 치료방법

첫째, 혈액 투석

둘째, 콩팥 이식 수술(일단 투석이 잘되고 있는 상태에서 가능합니다.).서명 : (인)

귀하의 경우 현재 다음과 같은 특이 사항이 있습니다.
복막투석

대한의학회 동의서식집 5

서약

위 말기(급성) 콩팥기능상실의 치료를 위한 복막 투석 처치에 따른 합병증에 대하여

충분히 이해하였으므로, 처치 중 합병증과 후유증 발생의 위험을 감수하고 본 처치를 받을

것을 신청합니다.""",
        """15-복막투석.pdf
복막유착 및 복막표면적의 상실로 부적절한 투석

라..투석 중 저혈압이 발생할 수 있습니다..서명 (인)

이 수술 이외의 치료방법

;

첫째, 혈액 투석

둘째, 콩팥 이식 수술(일단 투석이 잘되고 있는 상태에서 가능합니다.).서명 (인)

귀하의 경우 현재 다음과 같은 특이 사항이 있습니다.

서약

위 말기(급성) 콩팥기능상실의 치료를 위한 복막 투석 처리에 따른 합병증에 대하여

충분히 이해하였으므로, 처치 중 합병증과 후유증 발생의 위험을 감수하고 본 처치를 받을

것을 신청합니다.

위해서는 수술실에서 국소 마취 하에서 간단한 수술을 통하여

복강내에 복막 투석용 도관(카테터)을 삽입해야 합니다.

복막투석용 도관을 삽입한 후 정규적인 복막투석일정으로 들어갈 때까지
7˜14 일정도의 대기기간이 필요합니다.대기 기간이 지난 후 복막 투석을 시작합니다.자기 스스로 충분히

투석액 교환을 할 수 있다고 판단되면 퇴원을 할 수 있는데, 복막 투석용 도관 삽입 후

퇴원까지 10˜20 일 정도의 시일이 소요됩니다.그러나, 복막 투석 시작후 도관의

기능이 좋지 않아서 도관 삽입을 다시 시행할 경우도 있습니다.

복막에 정치한 복막카테터를 이용해서 투석액을 주입시킨 후에 일정시간이 지나면
배액시키고 새로운 투석액을 주입시키는 방법으로 일반적으로 6 시간마다 2L bag 을

사용하여 투석액을 교환하여 하루 4 회 교환하는 지속성 외래 복막 투석(CAPD,

Continuous Ambulatory Peritoneal Dialysis)이 가장 흔히 사용되는 방법이고, 환자의

상태나 복막의 기능에 따라 밤에 자는 동안 기계를 이용하여 복막투석을 하고 낮

동안에는 투석액을 복강내에 한번 넣은 후 지내는 자동화 복막투석(APD, Automated

Peritoneal analysis) 방법도 있습니다.

환자의 상태, 복막평형검사 결과 등에 따라 적절한 투석 방법을 사용합니다.복막투석의 적응증 및 금기증

적응 : 아래와 같은 경우 선호될 수 있는 것이지 절대적인 것은 아닙니다""",
    ]
)

scorer = Faithfulness(llm=evaluator_llm)
await scorer.single_turn_ascore(sample)



AttributeError: 'ChatCompletion' object has no attribute 'generate'

In [ ]:
import openai

# Azure OpenAI 설정
endpoint = 'https://3rdprjteam3eastus2.openai.azure.com/'
api_key = 'CYIGp1gg1utnMdZ73rRdELUZ9kbMyhhzBWzXQmG0IPbuMFRZzTQTJQQJ99BAACHYHv6XJ3w3AAABACOGdq4g'
deployment = "gpt-4o"

# Azure Ai Search 설정
search_endpoint = 'https://patient-faq-jb.search.windows.net'
search_key = 'ZOTDkign7PCfXVOkdgIGmRpWjTRSKl1YO6FEJfTfkjAzSeCW4TIk'
index_name = 'vector-total-3'

def chat_with_document():
    client = openai.AzureOpenAI(
        azure_endpoint=endpoint,
        api_key=api_key,
        api_version="2024-10-21",
    )
    '''
    답변엔 참조한 문서를 밝히지는 말고 질문에 관한 내용만 답변합니다.
    내가 요실금 수술을 받을건데 어떤 부작용이 있어?
    우리 할머니가 수술을 위해 마취를 하시는데, 혹시 생길 문제점이 있을까?
    복강경담낭절제수술의 증상은 무엇인가요?

    당신은 수술 동의서를 받은 환자들이 동의서를 잘 이해할 수 있도록 돕는 도우미입니다. \n\n 환자가 어떤 수술에 대해서 질문을 하면 가지고 있는 문서에서 파일명만을 검색해서 환자가 묻는 질문과 가장 유사한 제목의 문서를 참조해서 답변을 합니다. \n\n 만약 질문과 일치하는 문서명이 없다면 어떤 문서를 참조하면 좋을지 추론을 해보고 질문을 최대한 찾는 문서에 맞게 변경한 후 관련 문서를 통해 답변을 합니다. \n\n 질문에서 정보가 부족할 경우 사용자에게 정보를 더 얻을 수 있도록 요청을 합니다. \n\n 사용자가 법과 관련된 질문을 한다면 '의료법 일부.csv' 또는 '시행규칙 일부.csv'문서를 찾아보고 답변을 합니다. 참고한 내용이 몇조의 내용인지 확실하게 확인한 후에 각 답변마다 참고한 의료법 조항을 알려주고 그 조항의 내용도 알려줍니다. \n\n **관련 문서를 찾을 수 없더라도 계속 끝까지 시도를 합니다.**
    '''
    completion = client.chat.completions.create(
        model=deployment,
        messages=[
            {
                "role": "system",
                "content": "당신은 수술 동의서를 받은 환자들이 동의서를 잘 이해할 수 있도록 돕는 도우미입니다. \n\n 환자가 어떤 수술에 대해서 질문을 하면 가지고 있는 문서에서 파일명만을 검색해서 환자가 묻는 질문과 가장 유사한 제목의 문서를 참조해서 답변을 합니다. \n\n 만약 질문과 일치하는 문서명이 없다면 어떤 문서를 참조하면 좋을지 추론을 해보고 질문을 최대한 찾는 문서에 맞게 변경한 후 관련 문서를 통해 답변을 합니다. \n\n 질문에서 정보가 부족할 경우 사용자에게 정보를 더 얻을 수 있도록 요청을 합니다. \n\n 법률 정보는 의료법 문서나 시행규칙 문서에서 내용을 확인하여 답변을 합니다. 질문을 잘 추론해보고 의료법 문서의 내용에 맞게 답변을 합니다.\n\n **관련 문서를 찾을 수 없더라도 계속 끝까지 시도를 합니다.**"
            },
            {
                "role": "user",
                "content": "의료법에 따르면 수술 전 동의를 받아야하는 사항은 무엇이며, 이를 통해 환자의 권리 보호는 어떻게 보장되나요?",
            }
        ],
        extra_body={
            "data_sources":[
                {
                    "type": "azure_search",
                    "parameters": {
                        "endpoint": search_endpoint,
                        "index_name": index_name,
                        "query_type": "semantic",
                        "semantic_configuration": "vector-total-3-semantic-configuration",
                        "strictness" : 3,
                        "top_n_documents" : 5,
                        "authentication": {
                            "type": "api_key",
                            "key": search_key,
                        }
                    }
                }
            ],
        }
    )
    reply = completion.choices[0].message.content # AI 답변

    doc = completion.choices[0].message.context['citations'] # 참고 문서 내용
    content_dict = {item['title']: item['content'] for item in doc}

    intent = completion.choices[0].message.context['intent'] # 질문 의도
    print('답변 : ', reply)
    for i in range(len(doc)):
        print(doc[i]['title'])

    return reply, content_dict, intent

chat_with_document()

답변 :  의료법에 따르면, 수술 전 환자의 동의를 받아야 하는 사항은 다음과 같습니다 [doc1]:

1. 환자에게 발생하거나 발생 가능성이 있는 증상의 진단명.
2. 수술 등의 필요성, 방법 및 내용.
3. 환자에게 설명을 하는 의사, 치과의사 또는 한의사 및 수술 등에 참여하는 주된 의료인의 성명.
4. 수술 등에 따라 전형적으로 발생이 예상되는 후유증 또는 부작용.
5. 수술 전후 환자가 준비해야 할 사항.

이러한 동의 절차는 환자에게 충분한 정보를 제공하고, 환자가 자신의 건강과 관련된 결정을 내릴 수 있도록 돕는 데 목적이 있습니다. 이를 통해 환자의 권리 보호는 다음과 같은 방식으로 보장됩니다 [doc1]:

- 환자가 자신의 건강 상태와 치료 방법에 대해 충분히 이해하고, 자율적으로 선택할 수 있는 권리를 보장합니다.
- 의료진이 환자에게 필요한 정보를 명확히 제공함으로써, 환자의 알 권리를 충족시킵니다.
- 동의 과정에서 환자의 의사를 존중하고, 환자가 동의하지 않을 경우 강제하지 않음으로써 환자의 자기결정권을 보호합니다.

이러한 절차는 환자의 생명과 신체에 중대한 영향을 미칠 수 있는 의료 행위에서 환자의 권리를 최우선으로 고려하도록 설계되었습니다.
의료법.csv
시행규칙.csv
의료법.csv
의료법.csv
89-급성 경막하 혈종수술.pdf


('의료법에 따르면, 수술 전 환자의 동의를 받아야 하는 사항은 다음과 같습니다 [doc1]:\n\n1. 환자에게 발생하거나 발생 가능성이 있는 증상의 진단명.\n2. 수술 등의 필요성, 방법 및 내용.\n3. 환자에게 설명을 하는 의사, 치과의사 또는 한의사 및 수술 등에 참여하는 주된 의료인의 성명.\n4. 수술 등에 따라 전형적으로 발생이 예상되는 후유증 또는 부작용.\n5. 수술 전후 환자가 준비해야 할 사항.\n\n이러한 동의 절차는 환자에게 충분한 정보를 제공하고, 환자가 자신의 건강과 관련된 결정을 내릴 수 있도록 돕는 데 목적이 있습니다. 이를 통해 환자의 권리 보호는 다음과 같은 방식으로 보장됩니다 [doc1]:\n\n- 환자가 자신의 건강 상태와 치료 방법에 대해 충분히 이해하고, 자율적으로 선택할 수 있는 권리를 보장합니다.\n- 의료진이 환자에게 필요한 정보를 명확히 제공함으로써, 환자의 알 권리를 충족시킵니다.\n- 동의 과정에서 환자의 의사를 존중하고, 환자가 동의하지 않을 경우 강제하지 않음으로써 환자의 자기결정권을 보호합니다.\n\n이러한 절차는 환자의 생명과 신체에 중대한 영향을 미칠 수 있는 의료 행위에서 환자의 권리를 최우선으로 고려하도록 설계되었습니다.',
 {'의료법.csv': '「의료사고 피해구제 및 의료분쟁 조정 등에 관한 법률」 제6조에 따른 한국의료분쟁조정중재원이 의료분쟁의 조\r\n정 또는 중재 절차 개시 이후 환자 또는 환자 보호자의 동의를 받아 해당 업무의 수행을 위하여 요청하는 경우\r\n3.환자 및 해당 수술에 참여한 의료인 등 정보주체 모두의 동의를 받은 경우\r\n⑥ 누구든지 이 법의 규정에 따르지 아니하고 제2항에 따라 촬영한 영상정보를 탐지하거나 누출ㆍ변조 또는 훼손하\r\n여서는 아니 된다.\r\n⑦ 누구든지 제2항에 따라 촬영한 영상정보를 이 법에서 정하는 목적 외의 용도로 사용하여서는 아니 된다.\r\n⑧ 의료기관의 개설자는 보건복지부장관이 정하는 범위에서 제2항에 따라 촬영한 영상정보의 열람 